In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import numpy as np
from glob import glob
import upsilon
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import scipy.interpolate as interpolate
from scipy.optimize import curve_fit, least_squares
from statistics import mean, median
import pandas as pd
from dl import queryClient as qc
import collections
from astropy import stats
from astropy.table import Table
import os
import utils

In [3]:
if not os.path.exists('results2/plots'):
    os.makedirs('results2/plots')

In [29]:
def get_data(objname):
    df=qc.query(sql="""SELECT meas.* 
                     FROM nsc_dr2.meas
                     WHERE objectid='{:s}'""".format(objname),
              fmt='pandas',
              profile='db01')
    order = ['u','g','r','i','z']
    best_periods = []
    crv=[]
    fltrs=[]
    for f in order:
        selfltr = (df['filter'] == f)
        selfwhm = (df['fwhm'] <= 4.0)
        sel = selfltr & selfwhm
        t = df['mjd'][sel].values
        y = df['mag_auto'][sel].values
        dy = df['magerr_auto'][sel].values
        if len(t) < 25:
            continue

        best_periods.append(get_ls_period(t,y,objname=objname,f=f))
        crvi = np.vstack((t,y,dy)).T
        crv.append(crvi[np.argsort(crvi[:,0])])
        fltrs.append(f)
    period = 0
    for p in best_periods:
        period += p/len(best_periods)
    return crv, period, fltrs
        
def get_ls_period(t,y,min_freq=1./1.,max_freq=1./0.1,objname='_',f=''):
    """Use Lomb-Scargle periodogram to get an estimate on period"""
    
    ls = stats.LombScargle(t, y)
    frequency, power = ls.autopower(minimum_frequency=min_freq,maximum_frequency=max_freq)
    period = 1./frequency # period is the inverse of frequency
    
    best_period = period[np.argmax(power)]
    
#     plot_periodogram(period,power,best_period,objname=objname,f=f)
    return best_period

In [5]:
interesting = []
for obj in glob('interesting/*'):
    interesting.append(obj[12:-15])

In [6]:
t=Table([interesting],names=['id'])
t['period'] = -99.99
t['type']   = '          '
t['utyp']   = '          '
t['uprob']  = -99.99
t['uflag']  = -1
t['undat']  = 0
t['gtyp']   = '          '
t['gprob']  = -99.99
t['gflag']  = -1
t['gndat']  = 0
t['rtyp']   = '          '
t['rprob']  = -99.99
t['rflag']  = -1
t['rndat']  = 0
t['ityp']   = '          '
t['iprob']  = -99.99
t['iflag']  = -1
t['indat']  = 0
t['ztyp']   = '          '
t['zprob']  = -99.99
t['zflag']  = -1
t['zndat']  = 0

In [28]:
fdict = {'u':0,'g':4,'r':8,'i':12,'z':16}

t = Table.read("rrl_res_raw.fits",format='fits')
display(t)

# for n in range(len(interesting)):
#     crv,p_est,fltrs = get_data(interesting[n])
#     for i in range(len(fltrs)):
    
#         rf_model = upsilon.load_rf_model()
#         efeat = upsilon.ExtractFeatures(crv[i][:,0],crv[i][:,1],crv[i][:,2])
#         efeat.run()
#         feats = efeat.get_features()
#         rf_model = upsilon.load_rf_model()
#         label, probability, flag = upsilon.predict(rf_model, feats)
                
#         x = fdict[fltrs[i]]
#         t[n][t.colnames[3+x]] = label
#         t[n][t.colnames[4+x]] = probability
#         t[n][t.colnames[5+x]] = flag
#         t[n][t.colnames[6+x]] = len(crv[i])

#     uprob = t[n]['uprob']
#     utyp  = t[n]['utyp']
#     gprob = t[n]['gprob']
#     gtyp  = t[n]['gtyp']
#     rprob = t[n]['rprob']
#     rtyp  = t[n]['rtyp']
#     iprob = t[n]['iprob']
#     ityp  = t[n]['ityp']
#     zprob = t[n]['zprob']
#     ztyp  = t[n]['ztyp']
    
#     m = np.argmax(np.array([[uprob,gprob,rprob,iprob,zprob]]))
#     maxtyp = [utyp,gtyp,rtyp,ityp,ztyp][m]
    
#     t[n]['type'] = maxtyp

# import os

# if os.path.exists("rrl_res.fits"):
#     os.remove("rrl_res.fits")
# t.write("rrl_res.fits",format='fits')

id,period,type,utyp,uprob,uflag,undat,gtyp,gprob,gflag,gndat,rtyp,rprob,rflag,rndat,ityp,iprob,iflag,indat,ztyp,zprob,zflag,zndat
bytes13,float64,bytes10,bytes10,float64,int32,int32,bytes10,float64,int32,int32,bytes10,float64,int32,int32,bytes10,float64,int32,int32,bytes10,float64,int32,int32
100047_2267,-99.99,RRL_ab,RRL_ab,0.86,1,70,RRL_ab,0.95,0,258,RRL_ab,0.96,0,256,RRL_ab,0.97,0,267,RRL_ab,1.0,0,235
100413_6560,-99.99,EB_ED,DSCT,0.3,1,48,EB_ED,0.48,1,31,EB_ED,0.47,1,35,,-99.99,-1,0,,-99.99,-1,0
100826_1474,-99.99,CEPH_Other,,-99.99,-1,0,CEPH_Other,0.42,1,83,,-99.99,-1,0,,-99.99,-1,0,,-99.99,-1,0
100828_28520,-99.99,RRL_ab,,-99.99,-1,0,RRL_ab,0.8,1,91,,-99.99,-1,0,,-99.99,-1,0,,-99.99,-1,0
101068_6327,-99.99,EB_EC,EB_EC,0.69,1,32,EB_EC,0.64,1,40,EB_EC,0.63,1,49,EB_EC,0.43,1,54,EB_EC,0.58,1,57
101313_10442,-99.99,EB_EC,,-99.99,-1,0,EB_EC,0.85,1,1137,,-99.99,-1,0,,-99.99,-1,0,,-99.99,-1,0
101825_934,-99.99,RRL_ab,,-99.99,-1,0,RRL_ab,0.52,1,1130,,-99.99,-1,0,,-99.99,-1,0,,-99.99,-1,0
102337_12820,-99.99,EB_EC,,-99.99,-1,0,EB_EC,0.86,1,1141,,-99.99,-1,0,,-99.99,-1,0,,-99.99,-1,0
102338_5844,-99.99,DSCT,,-99.99,-1,0,DSCT,0.35,1,378,,-99.99,-1,0,,-99.99,-1,0,,-99.99,-1,0


In [9]:
tcf = Table.read("rrl_res_crvfit.fits",format='fits')
nomatch = np.argwhere(t['type']!=tcf['type'])

In [10]:
collections.Counter(t['type'])

Counter({'RRL_ab': 604,
         'EB_ED': 18,
         'CEPH_Other': 42,
         'EB_EC': 231,
         'DSCT': 86,
         'RRL_c': 121,
         'T2CEPH': 48,
         'NonVar': 4,
         'EB_ESD': 59,
         'CEPH_F': 39,
         'LPV_Mira_A': 1,
         'CEPH_1O': 36,
         'RRL_d': 1,
         'RRL_e': 4,
         'LPV_SRV_AG': 1})

In [11]:
ceph_names = []
for i in range(len(t)):
    if t[i]['type'] == 'CEPH_1O':
        ceph_names.append(t[i]['id'])

In [12]:
datpaths = glob('results2/*data.csv')
crvd = {}
for path in datpaths:
    if path[9:-11] not in crvd.keys():
        crvd[path[9:-11]] = {}
    crvd[path[9:-11]][path[-10:-9]] = np.genfromtxt(path,delimiter=',')

In [13]:
g = crvd[ceph_names[0]]['g']
r = crvd[ceph_names[0]]['r']

In [14]:
cf = {'u':u'#1f77b4','g':u'#2ca02c','r':u'#d62728','i':u'#9467bd','z':u'#8c564b'}

In [15]:
import csv
parampaths = glob('results2/*param.csv')
nms     = []
params  = []
pars    = []
periods = []
for path in parampaths:
    param = []
    fpars = []
    with open(path, newline='') as csvfile:
        csvreader = csv.reader(csvfile, delimiter=',')
        b     = True
        first = True
        for row in csvreader:
            if row[0] == '---':break
            if first:
                periods.append(float(row[2][2:]))
                first = False
                continue
            if b:
                param.append(row)
            else:
                fpars.append(row)
            b = not b
    param = np.array(param)
    fpars = np.array(fpars)
    nms.append(path[9:-10])
    params.append(param)
    pars.append(fpars)
nms = np.array(nms)
pdict = {nms[i]:periods[i] for i in range(len(nms))}

In [16]:
amps  = []
namps = []
for i in range(len(pars)):
    famps = []
    for j in range(len(pars[i])):
        famps.append(float(pars[i][j,1][4:]))
    amps.append(np.array(famps))
    
    namps.append(amps[-1]/np.mean(amps[-1]))

In [17]:
difamp  = []
sameamp = []
for i in range(len(namps)):
    if len(namps[i]) == 1:
        continue
    if np.std(namps[i]) > 0.1:
        difamp.append(i)
    else:
        sameamp.append(i)

In [18]:
difamp  = nms[difamp]
sameamp = nms[sameamp]

In [27]:
np.savetxt('difamp.txt',difamp,fmt='%s')
np.savetxt('sameamp.txt',sameamp,fmt='%s')

In [19]:
# import shutil
# for i in range(len(difamp)):
#     path = glob('results\{}*curve.png'.format(difamp[i]))[0]
#     shutil.copy(path,'results\dif amp')

In [20]:
# import shutil
# for i in range(len(sameamp)):
#     path = glob('results\{}*curve.png'.format(sameamp[i]))[0]
#     shutil.copy(path,'results\same amp')

In [26]:
crv,p_est,fltrs = get_data(gldrrab[0])
rf_model = upsilon.load_rf_model()
i=0
efeat = upsilon.ExtractFeatures(crv[i][:,0],crv[i][:,1],crv[i][:,2])
efeat.run()
feats = efeat.get_features()
rf_model = upsilon.load_rf_model()
label, probability, flag = upsilon.predict(rf_model, feats)

In [ ]:
t=Table([gldrrab],names=['id'])
t['period'] = -99.99
t['type']   = '          '
t['utyp']   = '          '
t['uprob']  = -99.99
t['uflag']  = -1
t['undat']  = 0
t['uprd']   = -99.99
t['gtyp']   = '          '
t['gprob']  = -99.99
t['gflag']  = -1
t['gndat']  = 0
t['gprd']   = -99.99
t['rtyp']   = '          '
t['rprob']  = -99.99
t['rflag']  = -1
t['rndat']  = 0
t['rprd']   = -99.99
t['ityp']   = '          '
t['iprob']  = -99.99
t['iflag']  = -1
t['indat']  = 0
t['iprd']   = -99.99
t['ztyp']   = '          '
t['zprob']  = -99.99
t['zflag']  = -1
t['zndat']  = 0
t['zprd']   = -99.99

In [72]:
fdict = {'u':0,'g':5,'r':10,'i':15,'z':20}
for n in range(0):#len(gldrrab)):
    crv,p_est,fltrs = get_data(gldrrab[n])
    periods = []
    for i in range(len(fltrs)):
    
        rf_model = upsilon.load_rf_model()
        efeat = upsilon.ExtractFeatures(crv[i][:,0],crv[i][:,1],crv[i][:,2])
        efeat.run()
        feats = efeat.get_features()
        rf_model = upsilon.load_rf_model()
        label, probability, flag = upsilon.predict(rf_model, feats)
                
        x = fdict[fltrs[i]]
        t[n][t.colnames[3+x]] = label
        t[n][t.colnames[4+x]] = probability
        t[n][t.colnames[5+x]] = flag
        t[n][t.colnames[6+x]] = len(crv[i])
        t[n][t.colnames[7+x]] = feats['period']
        periods.append(feats['period'])

    t[n]['period'] = (np.mean(periods))
    
    uprob = t[n]['uprob']
    utyp  = t[n]['utyp']
    gprob = t[n]['gprob']
    gtyp  = t[n]['gtyp']
    rprob = t[n]['rprob']
    rtyp  = t[n]['rtyp']
    iprob = t[n]['iprob']
    ityp  = t[n]['ityp']
    zprob = t[n]['zprob']
    ztyp  = t[n]['ztyp']
    
    m = np.argmax(np.array([[uprob,gprob,rprob,iprob,zprob]]))
    maxtyp = [utyp,gtyp,rtyp,ityp,ztyp][m]
    
    t[n]['type'] = maxtyp

In [73]:
t

id,period,type,utyp,uprob,uflag,undat,uprd,gtyp,gprob,gflag,gndat,gprd,rtyp,rprob,rflag,rndat,rprd,ityp,iprob,iflag,indat,iprd,ztyp,zprob,zflag,zndat,zprd
str13,float64,str10,str10,float64,int32,int32,float64,str10,float64,int32,int32,float64,str10,float64,int32,int32,float64,str10,float64,int32,int32,float64,str10,float64,int32,int32,float64
100047_2267,0.6221811344492266,RRL_ab,RRL_ab,0.86,1,70,0.6221598132668498,RRL_ab,0.95,0,258,0.6221890096889295,RRL_ab,0.96,0,256,0.6221758132967702,RRL_ab,0.97,0,267,0.6221889586416363,RRL_ab,1.0,0,235,0.6221920773519475
100555_10583,27.998177478696128,EB_ED,EB_ED,0.84,1,32,0.26383448636087176,EB_ED,0.6,1,43,0.3650679692453242,EB_ED,0.65,1,46,0.3650522258993856,EB_ED,0.6,1,56,0.3714944024749053,EB_ED,0.4,1,60,138.62543830950017
100828_28520,0.6283259392486608,RRL_ab,,-99.99,-1,0,-99.99,RRL_ab,0.8,1,91,0.6283259392486608,,-99.99,-1,0,-99.99,,-99.99,-1,0,-99.99,,-99.99,-1,0,-99.99
101825_934,0.6442550532084984,RRL_ab,,-99.99,-1,0,-99.99,RRL_ab,0.52,1,1130,0.6442550532084984,,-99.99,-1,0,-99.99,,-99.99,-1,0,-99.99,,-99.99,-1,0,-99.99
102849_2399,0.6488362241200318,RRL_ab,,-99.99,-1,0,-99.99,RRL_ab,0.69,1,1129,0.6488362241200318,,-99.99,-1,0,-99.99,,-99.99,-1,0,-99.99,,-99.99,-1,0,-99.99
102924_1422,1.7620586722694107,T2CEPH,,-99.99,-1,0,-99.99,DSCT,0.33,1,296,0.19415561818921295,,-99.99,-1,0,-99.99,T2CEPH,0.51,1,30,3.3299617263496084,,-99.99,-1,0,-99.99
105919_7718,1.2853866776261773,T2CEPH,,-99.99,-1,0,-99.99,T2CEPH,0.56,1,595,1.2853866776261773,,-99.99,-1,0,-99.99,,-99.99,-1,0,-99.99,,-99.99,-1,0,-99.99
106429_89,0.622850487439848,RRL_ab,,-99.99,-1,0,-99.99,RRL_ab,0.89,1,590,0.622850487439848,,-99.99,-1,0,-99.99,,-99.99,-1,0,-99.99,,-99.99,-1,0,-99.99
106432_8993,0.6359748424018525,RRL_ab,,-99.99,-1,0,-99.99,RRL_ab,0.96,1,1017,0.6359748424018525,,-99.99,-1,0,-99.99,,-99.99,-1,0,-99.99,,-99.99,-1,0,-99.99


In [74]:
import os
if os.path.exists("res.fits"):
    os.remove("res.fits")
t.write("res.fits",format='fits')

In [78]:
collections.Counter(t['type'])

Counter({'RRL_ab': 446,
         'EB_ED': 10,
         'T2CEPH': 33,
         'EB_EC': 17,
         'DSCT': 16,
         'CEPH_F': 18,
         'CEPH_Other': 17,
         'RRL_c': 15,
         'CEPH_1O': 5,
         'RRL_d': 2,
         'EB_ESD': 16})

In [79]:
446/595

0.7495798319327731

In [ ]:
def

In [33]:
gldorig = np.loadtxt('goldsample\golden_original.txt',delimiter=',',dtype=str)
gldrrab = np.loadtxt('goldsample\golden_RRab.txt',delimiter=',',dtype=str)
gldall  = np.union1d(gldorig,gldrrab)
gldelse = list(set(gldall) - set(gldrrab))

In [34]:
from astropy.table import Table
ampt=Table([gldelse],names=['id'])
ampt['uamp'] = -99.99
ampt['gamp'] = -99.99
ampt['ramp'] = -99.99
ampt['iamp'] = -99.99
ampt['zamp'] = -99.99

In [35]:
import csv
fdict = {'u':1,'g':2,'r':3,'i':4,'z':5}
for i,name in enumerate(gldelse):
    with open('results\{}_parameters.csv'.format(name), newline='') as csvfile:
        b,c,f = get_data(name)
        reader = csv.reader(csvfile, delimiter=',')
        first = True
        amps = []
        for row in reader:
            if first:
                first = False
                continue
            if row == ['---']:
                continue
            amps.append(float(row[1]))
        for j in range(len(f)):
            n = fdict[f[j]]
            ampt[i][n] = amps[j]

ampt.write("amp_else.fits",format='fits')

In [36]:
ampt

id,uamp,gamp,ramp,iamp,zamp
str13,float64,float64,float64,float64,float64
98403_9102,0.22,0.235,0.22,-99.99,0.21
192288_50605,-99.99,0.249,0.172,-99.99,-99.99
192287_129783,-99.99,0.268,0.167,-99.99,-99.99
167810_9877,-99.99,0.182,0.194,0.166,0.161
148116_39710,-99.99,0.514,0.4,0.215,0.242
150148_127174,-99.99,0.449,0.261,-99.99,-99.99
147561_59808,-99.99,0.086,0.056,0.077,0.076
147600_137355,-99.99,0.289,0.186,0.145,-99.99
149654_45283,-99.99,0.078,0.078,-99.99,-99.99


In [21]:
def

SyntaxError: invalid syntax (<ipython-input-21-7b18d017f89f>, line 1)

In [ ]:
figall, axall = plt.subplots()
figu, axu = plt.subplots()
figg, axg = plt.subplots()
figr, axr = plt.subplots()
figi, axi = plt.subplots()
figz, axz = plt.subplots()
for i in range(len(ceph_names)):
    p = pdict[ceph_names[i]]
    for fltr, crv in crvd[ceph_names[i]].items():
        axall.scatter(crv[:,0]/p,crv[:,1]/max(crv[:,1]),s=1)
        if fltr == 'u':
            axu.scatter(crv[:,0]/p,crv[:,1]/max(crv[:,1]),s=1)
        if fltr == 'g':
            axg.scatter(crv[:,0]/p,crv[:,1]/max(crv[:,1]),s=1)
        if fltr == 'r':
            axr.scatter(crv[:,0]/p,crv[:,1]/max(crv[:,1]),s=1)
        if fltr == 'i':
            axi.scatter(crv[:,0]/p,crv[:,1]/max(crv[:,1]),s=1)
        if fltr == 'z':
            axz.scatter(crv[:,0]/p,crv[:,1]/max(crv[:,1]),s=1)

In [ ]:
shortlist = []
for name, p in pdict.items():
    if name in ceph_names:
        shortlist.append(name)

In [ ]:
coeffs = np.polyfit(crv[:,0],crv[:,1],1)
coeffs

In [ ]:
x = np.arange(len(coeffs))
y = np.array([np.sum(np.array([coeffs[i]*(j**i) for i in range(len(coeffs))])) for j in x])

In [ ]:
ceph_inds = []
for i,name in enumerate(t['id']):
    if name in ceph_names:
        ceph_inds.append(i)

In [ ]:
t[ceph_inds]

In [ ]:
for i in range(len(ceph_names)):
    if ceph_names[i] in ['189344_10414','190269_170897','191137_70691','191515_128100','151560_15723','192287_129783']:
        print(i)

In [ ]:
phshft = np.zeros(36)
phshft[0]  += .02
phshft[11] += .025
phshft[23] += .025
phshft[25] += .035
phshft[28] += .02
phshft[32] -= .02

amp = np.ones(36)
amp[0]  = 1.05
amp[9]  = 0.95
amp[19] = 1.08
amp[20] = 1.05
amp[12] = 1.25
amp[22] = 1.7

yoffset = np.zeros(36)
yoffset[14] = -.175

In [ ]:
fig, ax = plt.subplots(36,figsize=(15,150))
count = 0
tt = []
yy = []
plist = []
for i in range(36):
    p = pdict[ceph_names[i]]
    plist.append(p)
    
    if 'g' in crvd[ceph_names[i]].keys():
        crv = crvd[ceph_names[i]]['g']
        tt.append((crv[:,0]/p+phshft[i])%1)
        yy.append(crv[:,1]/max(crv[:,1]))
        count = count + 1
        ax[i].scatter((crv[:,0]/p+phshft[i])%1,amp[i]*crv[:,1]/max(crv[:,1]),label=i,s=20)
        ax[i].set_ylim(-.1,1.1)
    
    
    for j in [25,26,28,29,32]:
        pj = pdict[ceph_names[j]]
        crvj = crvd[ceph_names[j]]['g']
        ax[i].scatter((crvj[:,0]/pj+phshft[j])%1,amp[j]*crvj[:,1]/max(crvj[:,1]),label=j,s=2)

    ax[i].legend()

[35] no g<br>
[34,15,26,28,31],[9?,18?,23?,25?,29?]<br>
[33,9,10,11,19,27,28],[2?,4?,23?,29?]<br>
[32,29],[13?,17?,18?,28?,30?],[~5,~6,~7]<br>
[31,9,,10,26,34],[28?]<br>
[30,15,25,26,32],[2?,7?,17?,18?]<br>
[29,0,15,25,26,28],[6?,7?,17?,18?,32?,34?,30?]<br>
[28,19,26,29,31],[7?,15?,25?,34?]<br>
[27,19],[1?,4?,11?]<br>
[26,18,28,29,32],[13?,17?,25?]
[25,15],[0?,14?,17?,18?,26?,28?,29?]<br>
[24] no g<br>
[23] Left side is off<br>
[22,5,20],[1?,6?,7?,13?]<br>
[21] Bad<br>
[20,5,22],[13?]<br>
[19,2,9,10],[11?,33?]<br>
[18,9,13,17,26,29],[6?,7?,15?,22,25?,28?]<br>
[17,13,18,22,26,32],[5?,6?,20?,25?,28?,29?,30?]<br>
[16,4,11,12],[27?]<br>
[15,0,7,25,26,28,29,30,32],[14?,18?,31?,34?]<br>
[14,0],[25?,28?,29?]<br>
[13,1,5,6,17,18,20,22,25,26,29,32],[7?,28?,30?,31?]<br>
[12,2,16,9,19,33],[4?,8?,10?,11?]<br>
[11,9,10,19,33],[2?,4?,16?,23?,27?]<br>
[10,2,9,11,19,22,31],[13?]<br>
[9,2,11,19,22,23,26,28,29,33,34]<br>
[8] Bad<br>
[7,0,6,15,22,29]<br>
[6,0,7,13,28],[20?,22?]<br>
[5,1,6,13,20,27],[22?]<br>
[4,9,10,11,12,16,19,27,33]<br>
[3]<br>
[2,4,9,10,11,19],[23?]<br>
[1,5,20,27]<br>
[0,6,7,14,25,28,29,32]<br>
RRLab?   t [17,18]<br>
irr [3,21,8]<br>

[17,13,18,22,26,32]<br>
13: good on right side, bad in middle, maybe RRL ab?<br>
17: Looks like RRL ab<br>
22: Messy, hard to tell amplitude due to outlier

In [ ]:
fig, ax = plt.subplots(2,figsize=(10,7))
for i in [25,26,28,29,32]:
    if 'g' in crvd[ceph_names[i]].keys():
        crv = crvd[ceph_names[i]]['g']
        p = pdict[ceph_names[i]]
        tt.append((crv[:,0]/p+phshft[i])%1)
        yy.append(crv[:,1]/max(crv[:,1]))
        size=5
        ax[0].scatter((crv[:,0]/p+phshft[i])%1,amp[i]*crv[:,1]/max(crv[:,1])+yoffset[i],label=i,s=size)
        ax[1].scatter((crv[:,0]/p+phshft[i])%1,amp[i]*crv[:,1]/max(crv[:,1])+yoffset[i],label=i,s=2,c='black')

for j in []:
    if 'g' in crvd[ceph_names[j]].keys():
        crv = crvd[ceph_names[j]]['g']
        p = pdict[ceph_names[j]]
        tt.append((crv[:,0]/p+phshft[j])%1)
        yy.append(crv[:,1]/max(crv[:,1]))
        ax[1].scatter((crv[:,0]/p+phshft[j])%1,amp[j]*crv[:,1]/max(crv[:,1])+yoffset[j],label=j,s=30)

ax[0].set_ylim(-.1,1.1)
ax[0].legend()
ax[1].set_ylim(-.1,1.1)
ax[1].legend()

In [ ]:
fig, ax = plt.subplots(2,figsize=(10,7))
for i in [2,9,10,19]:#[2,9,10,11,19]:
    if 'g' in crvd[ceph_names[i]].keys():
        crv = crvd[ceph_names[i]]['g']
        p = pdict[ceph_names[i]]
        tt.append((crv[:,0]/p+phshft[i])%1)
        yy.append(crv[:,1]/max(crv[:,1]))
        size=5
        ax[0].scatter((crv[:,0]/p+phshft[i])%1,amp[i]*crv[:,1]/max(crv[:,1])+yoffset[i],label=i,s=size)
        ax[1].scatter((crv[:,0]/p+phshft[i])%1,amp[i]*crv[:,1]/max(crv[:,1])+yoffset[i],label=i,s=2,c='black')

for j in [11]:
    if 'g' in crvd[ceph_names[j]].keys():
        crv = crvd[ceph_names[j]]['g']
        p = pdict[ceph_names[j]]
        tt.append((crv[:,0]/p+phshft[j])%1)
        yy.append(crv[:,1]/max(crv[:,1]))
        ax[1].scatter((crv[:,0]/p+phshft[j])%1,amp[j]*crv[:,1]/max(crv[:,1])+yoffset[j],label=j,s=5)

ax[0].set_ylim(-.1,1.1)
ax[0].legend()
ax[1].set_ylim(-.1,1.1)
ax[1].legend()

In [ ]:
for i in [25,26,28,29,32]:
    crv = crvd[ceph_names[i]]['g']
    print("{:2}: {:.3f}, {:.4f}".format(i,plist[i],max(crv[:,1])-min(crv[:,1])))

In [ ]:
for j in [0,15,31,34]:
    crv = crvd[ceph_names[j]]['g']
    print("{:2}: {:.3f}, {:.4f}".format(j,plist[j],max(crv[:,1])-min(crv[:,1])))

In [ ]:
ceph_names[34]

In [ ]:
crv = crvd[ceph_names[15]]['g']
plt.scatter(crv[:,0],crv[:,1])

In [ ]:
mcrv = np.vstack((tt,yy)).T
crv1 = mcrv[mcrv[:,0].argsort()]
#bspline fit
t,c,k = interpolate.splrep(crv1[:,0],crv1[:,1],s=5,k=5)
spline = interpolate.BSpline(t,c,k)
plt.scatter(crv1[:,0],crv1[:,1],s=3)
plt.plot(crv1[:,0],spline(crv1[:,0]),c='k')

scipy.interpolate.LSQUnivariateSpline

#10 has good coverage but too wide
#12 and #13 #14 #15 #5 #17

short:
#3, #16

bad period?
#18 #6

In [ ]:
# figall, axall = plt.subplots()
figg, axg = plt.subplots()
# figr, axr = plt.subplots()
# figi, axi = plt.subplots()
# figz, axz = plt.subplots()
count = 0
phshft = np.zeros(20)
for i in [4,9,10,11,12,16,19,27,33]:
    p = pdict[ceph_names2[i]]
    print((i,p))
    for fltr, crv in crvd[ceph_names2[i]].items():
        m = int(len(crv)*0)
        n = int(len(crv)*1)
#         axall.scatter(crv[m:n,0]/p,crv[m:n,1]/max(crv[:,1]))
        if fltr == 'g':
            count = count + 1
            axg.scatter(crv[m:n,0]/p+phshft[i],crv[m:n,1]/max(crv[:,1]),label=i,s=5)
axg.legend()
#         if fltr == 'r':
#             axr.scatter(crv[m:n,0]/p,crv[m:n,1]/max(crv[:,1]))
#         if fltr == 'i':
#             axi.scatter(crv[m:n,0]/p,crv[m:n,1]/max(crv[:,1]))
#         if fltr == 'z':
#             axz.scatter(crv[m:n,0]/p,crv[m:n,1]/max(crv[:,1]))
#             coeffs = np.polyfit(crv[m:n,0],crv[m:n,1],2)
#             y = np.array([np.sum(np.array([coeffs[i]*(j**i) for i in range(len(coeffs))])) for j in crv[m:n,0]])
#             #axz.plot(y,crv[m:n,0])

In [ ]:
mcrv = np.vstack((t2,y2)).T
crv1 = mcrv[mcrv[:,0].argsort()]
#bspline fit
t,c,k = interpolate.splrep(crv1[:,0],crv1[:,1],s=1.1,k=5)
spline = interpolate.BSpline(t,c,k)
plt.scatter(crv1[:,0],crv1[:,1],s=3)
plt.plot(crv1[:,0],spline(crv1[:,0]),c='k')

In [ ]:
figall, axall = plt.subplots()
figu, axu = plt.subplots()
figg, axg = plt.subplots()
figr, axr = plt.subplots()
figi, axi = plt.subplots()
figz, axz = plt.subplots()
for i in range(len(ceph_names)):
    p = pdict[ceph_names[i]]
    fig1, ax1 = plt.subplots()
    for fltr, crv in crvd[ceph_names[i]].items():
        axall.scatter(crv[:,0]/p,crv[:,1]/max(crv[:,1]))
        ax1.scatter(crv[:,0]/p,crv[:,1]/max(crv[:,1]),c=cf[fltr],label=fltr)
        if fltr == 'u':
            axu.scatter(crv[:,0]/p,crv[:,1]/max(crv[:,1]))
        if fltr == 'g':
            axg.scatter(crv[:,0]/p,crv[:,1]/max(crv[:,1]))
        if fltr == 'r':
            axr.scatter(crv[:,0]/p,crv[:,1]/max(crv[:,1]))
        if fltr == 'i':
            axi.scatter(crv[:,0]/p,crv[:,1]/max(crv[:,1]))
        if fltr == 'z':
            axz.scatter(crv[:,0]/p,crv[:,1]/max(crv[:,1]))
    ax1.set_title('{} - Phased Normalized'.format(ceph_names[i]))
    ax1.legend()
    fig1.savefig('overplot/{}_phased_norm.pdf'.format(ceph_names[i]))
    plt.close(fig1)
axu.set_title('u - Phased Normalized')
figu.savefig('overplot/u_phased_norm.pdf')
plt.close(figu)
axg.set_title('g - Phased Normalized')
figg.savefig('overplot/g_phased_norm.pdf')
plt.close(figg)
axr.set_title('r - Phased Normalized')
figr.savefig('overplot/r_phased_norm.pdf')
plt.close(figr)
axi.set_title('i - Phased Normalized')
figi.savefig('overplot/i_phased_norm.pdf')
plt.close(figi)
axz.set_title('z - Phased Normalized')
figz.savefig('overplot/z_phased_norm.pdf')
plt.close(figz)
axall.set_title('All - Phased Normalized')
figall.savefig('overplot/all_phased_norm.pdf')
plt.close(figall)